In [98]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from metpy.calc import pressure_to_height_std as press_to_alt
from metpy.units import units
import pandas as pd
import narpy as na
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
%matplotlib notebook

In [7]:
radar_dir = "data/wind/radar/capel-dewi/"
forcast_dir = "data/wind/gfs/0p25/capel-dewi/"
ground_dir="data/wind/ground/frongoch/"

radar_lat=52.42
radar_long=-4.01
sample_point=[('latitude', radar_lat),('longitude', radar_long)]

In [8]:
def daterange(start_date, end_date):
#https://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python
    for n in range(int((end_date - start_date).days+1)):
        yield start_date + timedelta(n)

In [9]:
def purge_unreliable(data,alt,reliable):
    data_clean,alt_clean=[],[]
    for ind,rel in enumerate(reliable):
        if rel==1:
            data_clean.append(data[ind])
            alt_clean.append(alt[ind])
    
    return data_clean,alt_clean

In [10]:
start_date="20190801"
end_date="20190810"

start_dt = datetime.strptime(start_date, "%Y%m%d")
end_dt = datetime.strptime(end_date, "%Y%m%d")

runs=["0000","0600","1200","1800"]
forcast_distance="003"

In [12]:
radar1=nc.Dataset("%s%s.nc"%(radar_dir,start_date))
forcast1=nc.Dataset("%s%s.nc"%(forcast_dir,start_date))
ground1=na.file("%s%s.na"%(ground_dir,start_date))

In [87]:
radar1

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.0
    title: 46.5 MHz wind-profiling radar Cartesian data - st300 mode
    institution:   Data recorded by the Natural Environment Research Council (NERC)
Mesosphere-Stratosphere-Troposphere (MST) Radar Facility at Aberystwyth
- http://mst.nerc.ac.uk
  Data processed by the Rutherford Appleton Laboratory, Space Science and
Technology Department - http://www.sstd.rl.ac.uk
  Data held at the British Atmospheric Data Centre
http://badc.nerc.ac.uk/data/mst

    source:   The Natural Environment Research Council (NERC) Mesosphere-Stratosphere-
Troposphere (MST) Radar at Aberystwyth

    history: File created 2019-08-02 03:03:23 +00:00 on machine claudius
2019-08-02T13:02:21 - Data passed visual quality control check and file
  delivered to CEDA for ingestion into archives.
    references:   Basic information about the data is available at 
http://badc.nerc.ac.uk/data/mst
  

In [212]:
forcast1["time"][:]

masked_array(data=[ 0,  3,  6,  9, 12, 15, 18, 21],
             mask=False,
       fill_value=999999,
            dtype=int32)

In [209]:
def nearest(search_list, value):
    val=sorted(search_list, key=lambda x: abs(float(x)-float(value)), reverse=True)[-1]
    return search_list.index(val),val

In [175]:
forcast_n_wind=np.array(forcast1.variables["VWND"][1]).flatten()
forcast_alt=np.array(press_to_alt(units.hPa*np.array(forcast1.variables["PRES"][1]).flatten())*1000)

In [187]:
radar_t_ind,t=nearest(list(radar1["time"][:]),3*60*60)
ground_t_ind,t=nearest(ground1["Seconds since 2019/08/01 00:00:00 UTC at start of 1 minute sample period (s)"].data,3*60*60)
obs_n_wind,obs_alt=purge_unreliable(np.array(list(radar1["northward_wind"][radar_t_ind])+[ground1["Mean northward wind in 1 minute sample period (m s-1)"].data[ground_t_ind]]),
                                        np.array(list(radar1["altitude"])+[140+10]),
                                        np.array(list(radar1["horizontal_wind_components_are_reliable"][radar_t_ind])+[1]))
diff_n_wind,diff_alt=difference(forcast_alt,forcast_n_wind,obs_alt,obs_n_wind)

In [188]:
sorted(diff_alt)[0]

150.0

In [388]:
N=15
x_min=0
x_max=20000
mu_base=list(np.linspace(x_min,x_max,N))
sig_0_base=list(((x_max-x_min)/N)*np.ones(N)*1.5)
a_0_base=list(np.ones(N))


results_a=[]
results_sig=[]
for time in ["3"]:#,"9","15","21"]:
    forcast_t_ind,t=nearest([ 0,  3,  6,  9, 12, 15, 18, 21],time)
    radar_t_ind,t=nearest(list(radar1["time"][:]),int(time)*60*60)
    ground_t_ind,t=nearest(ground1["Seconds since 2019/08/01 00:00:00 UTC at start of 1 minute sample period (s)"].data,int(time)*60*60)
    
    forcast_n_wind=np.array(forcast1.variables["VWND"][forcast_t_ind]).flatten()
    forcast_alt=np.array(press_to_alt(units.hPa*np.array(forcast1.variables["PRES"][forcast_t_ind]).flatten())*1000)
    obs_n_wind,obs_alt=purge_unreliable(np.array(list(radar1["northward_wind"][radar_t_ind])+[ground1["Mean northward wind in 1 minute sample period (m s-1)"].data[ground_t_ind]]),
                                        np.array(list(radar1["altitude"])+[140+10]),
                                        np.array(list(radar1["horizontal_wind_components_are_reliable"][radar_t_ind])+[1]))
    
    diff_n_wind,diff_alt=difference(forcast_alt,forcast_n_wind,obs_alt,obs_n_wind)
    
    mu=[]
    sig_0=[]
    a_0=[]
    for ind,m in enumerate(mu_base):
        if sorted(diff_alt)[0]<m<sorted(diff_alt)[-1]:
            mu.append(m)
            sig_0.append(sig_0_base[ind])
            a_0.append(a_0_base[ind])
            
    N=len(mu)
    """params_0=np.ravel(np.array(sig_0+a_0))
    
    popt,pcov=curve_fit(wrapper_guass,np.array(diff_alt),diff_n_wind,p0=[params_0],maxfev=10000,sigma=np.array([3]*len(diff_alt)))
    plt.plot(np.linspace(150-1000,20000,N*100),wrapper_guass(np.linspace(150-1000,20000,N*100),list(popt)))
    plt.errorbar(diff_alt,diff_n_wind,yerr=3,marker="x")
    #plt.plot(np.linspace(150-1000,20000,N*100),wrapper_guass(np.linspace(150-1000,20000,N*100),list(params_0)))
    results_sig.append(popt[:int(len(popt)/2)])
    results_a.append(popt[int(len(popt)/2):])"""
    sig=sig_0
    popt,pcov=curve_fit(wrapper_guass_a,np.array(diff_alt),diff_n_wind,p0=a_0,maxfev=10000)
    plt.plot(np.linspace(x_min,x_max,N*100),wrapper_guass_a(np.linspace(x_min,x_max,N*100),list(popt)))
    plt.errorbar(diff_alt,diff_n_wind,yerr=3,marker="x")
    
N=9
plt.xlim(0)

<IPython.core.display.Javascript object>

(0.0, 21000.0)

In [344]:
results_sig=pd.DataFrame(results_sig)
for col in results_sig.columns:
    results_sig[col].fillna(value=results_sig[col].mean(), inplace=True)
results_a=pd.DataFrame(results_a)
for col in results_a.columns:
    results_a[col].fillna(value=results_a[col].mean(), inplace=True)

In [354]:
means_sig=np.array(results_sig.mean())
cov_sig=np.array(results_sig.cov())
random_sig=np.random.multivariate_normal(means_sig,cov_sig)
means_a=np.array(results_a.mean())
cov_a=np.array(results_a.cov())
random_a=np.random.multivariate_normal(means_a,cov_a)

print(random_a,random_sig)

[  0.45492971   5.16389592  -5.83459726 -18.69628097   6.79071633
   9.59547564  -2.00400681   1.12851864] [15994.98844869  1783.88338378   241.62492441  3093.34615603
  -547.10266234  7590.73826398   372.49831079    33.78949304]


In [355]:
plt.plot(np.linspace(150-1000,20000,N*100),wrapper_guass(np.linspace(150-1000,20000,N*100),list(list(random_sig)+list(random_a))),"--")

In [92]:
def difference(forcast_alt,forcast_wind,obs_alt,obs_wind):
    forcast=interp1d(forcast_alt,forcast_wind)
    diff_wind,diff_alt=[],[]
    for ind,alt in enumerate(obs_alt):
        diff_wind.append(forcast(alt)-obs_wind[ind])
        diff_alt.append(alt)
    return diff_wind,diff_alt

In [99]:
def guass(x,mu,sigma,a):
    return a*np.exp(-(x-mu)**2/(2*sigma**2))

def guass_proc(x,sig,a):
    y=np.zeros(len(x))
    for ind,m in enumerate(mu):
        y_n=guass(x,m,sig[ind],a[ind])
        y+=y_n
    return y

def wrapper_guass(x,*args):
    if len(args)==1:
        sig,a=args[0][:N],args[0][N:2*N]
    else:
        args=list(args)
        sig,a=args[:N],args[N:2*N]

    return guass_proc(x,sig,a)

In [361]:
def guass_proc_a(x,a):
    y=np.zeros(len(x))
    for ind,m in enumerate(mu):
        y_n=guass(x,m,sig[ind],a[ind])
        y+=y_n
    return y

def wrapper_guass_a(x,*args):
    if len(args)==1:
        a=args[0][:N]
    else:
        args=list(args)
        a=args[:N]

    return guass_proc_a(x,a)

In [83]:
ground=pd.read_csv("%s%s.txt"%(ground_dir,"2020"),names=["datetime","type","id","hour_count","name","verison","src_id","state","mean_wind_dir","mean_wind_speed","max_gust_dir","max_gust_speed","max_gust_ctime","mean_wind_dir_q","mean_wind_speed_q","max_gust_dir_q","max_gust_speed_q","mac_gust_ctime_q","meto_stmp_time","mindas_stmp_time","j1","j2","j3","j4"])

In [87]:
ground.src_id.unique()

array([61974, 25727,     3,     9,    12,    23,    32,    44,    48,
          52,    54, 30270,    67,    79, 18903,   105,   113,   117,
         132,   137,   145,   150,   161,   170,   177, 17336,   212,
         235, 19260,   268, 16589, 30523,   315, 16596, 17314,   346,
       17344,   358,   370,   373,   381,   384,   386,   393,   395,
       16725,   405,   409, 61986,   421,   440,   456,   461, 19188,
         498,   513, 56958,   534,   554,   556,   583,   595, 19187,
       24102,   605,   613, 17176, 62083, 30690,   643,   657, 24996,
         669,   671,   674, 62122,   692, 62084, 62073,  6182,   708,
         709,   719,  6313,   723, 30620,   726,   743,   744,   775,
         779,   795,   811,   842,   847, 56962,   862,   869, 17224,
         876,   886,   888,   889, 18974, 23417,   908,   918, 24125,
         982,   987,  1007,  1023,  1033,  1039,  1046,  1055,  1060,
        1074,  1076,  1078,  1083,  1085,  1090, 17309,  1125, 57199,
        1137,  1144,

In [89]:
radar1

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.0
    title: 46.5 MHz wind-profiling radar Cartesian data - st300 mode
    institution:   Data recorded by the Natural Environment Research Council (NERC)
Mesosphere-Stratosphere-Troposphere (MST) Radar Facility at Aberystwyth
- http://mst.nerc.ac.uk
  Data processed by the Rutherford Appleton Laboratory, Space Science and
Technology Department - http://www.sstd.rl.ac.uk
  Data held at the British Atmospheric Data Centre
http://badc.nerc.ac.uk/data/mst

    source:   The Natural Environment Research Council (NERC) Mesosphere-Stratosphere-
Troposphere (MST) Radar at Aberystwyth

    history: File created 2020-08-02 03:00:39 +00:00 on machine claudius
2020-08-03T08:32:01 - Data passed visual quality control check and file
  delivered to CEDA for ingestion into archives.
    references:   Basic information about the data is available at 
http://badc.nerc.ac.uk/data/mst
  